In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from normalize import clean_annotations, normalize_annots

AssertionError: could not find onto_root at /resources/bluima/neuroner/

In [2]:
from neuroner.normalize import clean_annotations, normalize_annots

ImportError: No module named normalize

In [ ]:
from similarity import similarity
from similarity_intra import load_ontologies

# Computes the intra and inter semantic similarity between two neurons
# in: n1@str, n2@str: the two neurons to measure similarity
# out: (score:float, [(matching_properties, explanation@str)])
#similarity('PV Layer 1 cortex acetylcholine neuron', 'Layer 1a striatum gaba fast-spiking neuron')
similarity('Periventricular zone layer 5a neuron', 'Hypothalamic medial zone layer 5-6 neuron')

In [ ]:
import pandas as pd


# download neuroelectro summary spreadsheet from URL and import into pandas data frame
neuroelectro_dataset_url = "http://dev.neuroelectro.org/static/src/article_ephys_metadata_curated.csv"
table = pd.read_csv(neuroelectro_dataset_url, sep='\t', index_col = 0, header=0)

In [ ]:
big_onto = load_ontologies()

In [ ]:
name_spaces = [('NCBI_GENE', 'gene'), ('HBP_LAYER', 'layer'), ('HBP_MORPHOLOGY', 'shape'), ('REGION', 'region'),
              ('HBP_EPHYS', 'ephys'), ('MOUSE_LINE', 'line'),]
proj_name = 'projection'

proj_list = ['CStr', 'CSp', 'CThal', 'CCo', 'CTrig', 'CTect']

def annots_to_dict(annot_list):
    temp_dict = {}
    for annotation in annot_list:
        for s in name_spaces:
            if s[0] in annotation:
                term = _normalize([annotation], shorten = True)
                if term in proj_list:
                    temp_dict[proj_name] = term
                    continue
                if s[1] not in temp_dict:
                    temp_dict[s[1]] = term
                else:
                    temp_dict[s[1]] = '%s %s' %  (temp_dict[s[1]],  term)
    return temp_dict

In [ ]:
strain_list = ['GIN', 'G42', 'GAD65-GFP', 'Thy1-YFP']
def add_animal_strain_to_neuron(input_neuron_name, strain):
    output_neuron_name = input_neuron_name

    if strain in strain_list:
        #output_neuron_name += ' from %s mouse line' % strain
        output_neuron_name += ' %s' % strain
    return output_neuron_name
def check_strain(neuron_name):
    for s in strain_list:
        if s in neuron_name:
            return 'MOUSE_LINE:%s' % s
    return None
check_strain('Hypothalamic medial zone dopaminergic neuron from Thy1-YFP mouse line')

In [ ]:
from sherlok import Sherlok
s = Sherlok('neuroner')

from similarity import _cleanup, _normalize

neuron_name = 'Neocortex corticostriatal NPY somatostatin-expressing with depolarizing fast spiking AHP Layer 5a pyramidal cell GIN'
r = s.annotate(neuron_name)
#annot_list = format_annots(neuron_name, r.annotations)
al = _cleanup(r.annotations, neuron_name)
if check_strain(neuron_name):
    al.append(check_strain(neuron_name))
print _normalize(al, shorten = True)

#print annot_list
print al

In [ ]:
# use sherlok to annotate each curated neuron mention in neuroelectro
annot_vec = []
dict_list = []
for index, row in table.iterrows():
    neuron_name_orig = row['NeuronLongName']
    strain = unicode(row['Strain'])
    # if no neuron long name, use neurolex neuron name (should always be available)
    if pd.isnull(neuron_name_orig):
        neuron_name_orig = row['NeuronName']
        
    # hack for adding strain annotations, since not in neuroNER
    neuron_name = add_animal_strain_to_neuron(neuron_name_orig, strain)
    
    # annotate using sherlok
    r = s.annotate(neuron_name)
    cleaned_annotations = _cleanup(r.annotations, neuron_name)
    
    if check_strain(neuron_name):
        cleaned_annotations.append(check_strain(neuron_name))
        for c in cleaned_annotations:
            if c.startswith('NCBI_GENE'):
                cleaned_annotations.remove(c)
    short_annots = _normalize(cleaned_annotations, shorten = True)
    temp_dict = annots_to_dict(cleaned_annotations)
    temp_dict['neuron_name'] = neuron_name
    temp_dict['neuron_name_orig'] = neuron_name_orig
    temp_dict['short_name'] = short_annots
    temp_dict['annotations'] = cleaned_annotations
    temp_dict['Index'] = index
    dict_list.append(temp_dict)

#     f = format_annots(neuron_name, r.annotations)
#     annot_vec.append((neuron_name, f))

In [ ]:
dict_list[4]

In [ ]:
col_names = [n[1] for n in name_spaces]
col_names.append('projection')

df = pd.DataFrame.from_records(dict_list, index = 'Index')
df

table[0:4]

In [ ]:
table = table.join(df)

In [ ]:
table.to_csv('df.csv', sep='\t', encoding='utf-8')

In [ ]:
table.head()

In [ ]:
# load in csv of Design.csv dataset

gene_exp_path = '/home/stripathy/brainCellTypeSpecificGenes/data/Design.tsv'
gene_exp_table = pd.read_csv(gene_exp_path, sep='\t', index_col = 0, header=0)

In [ ]:
gene_exp_table.tail()

In [ ]:
import re

# use sherlok to annotate each curated neuron mention in neuroelectro
#annot_vec = []
gene_dict_list = []
for index, row in gene_exp_table.iterrows():
    neuron_name = row['neuroNER Inferred Properties']
    
    # if no neuron long name, just move on
    if pd.isnull(neuron_name):
        temp_dict = {}
        temp_dict['annotations'] = []
        temp_dict['Index'] = index
        temp_dict['short_name'] = ''
        temp_dict['neuron_name'] = ''
        gene_dict_list.append(temp_dict)
        continue
        # neuron_name = row['NeuronName']
    
    mouse_line_match = re.search('from .+ mouse line', neuron_name)
    if mouse_line_match:
        strain_annotation = check_strain(neuron_name)
        neuron_name = re.sub('from .+ mouse line', '', neuron_name)
    
    # annotate using sherlok
    r = s.annotate(neuron_name)
    cleaned_annotations = _cleanup(r.annotations, neuron_name)
    if mouse_line_match:
        cleaned_annotations.append(strain_annotation)
        for c in cleaned_annotations:
            if c.startswith('NCBI_GENE'):
                cleaned_annotations.remove(c)
    short_annots = _normalize(cleaned_annotations, shorten = True)
    temp_dict = annots_to_dict(cleaned_annotations)
    temp_dict['short_name'] = short_annots
    temp_dict['neuron_name'] = neuron_name
    temp_dict['annotations'] = cleaned_annotations
    temp_dict['Index'] = index
    gene_dict_list.append(temp_dict)

#     f = format_annots(neuron_name, r.annotations)
#     annot_vec.append((neuron_name, f))

In [ ]:
df_gene = pd.DataFrame.from_records(gene_dict_list, index = 'Index')
gene_exp_table = gene_exp_table.join(df_gene)

In [ ]:
gene_exp_table.to_csv('df_gene.csv', sep='\t', encoding='utf-8')

In [ ]:
for index, row in gene_exp_table.iterrows():
    curr_list = row
    sim_vec = []
    sim_val_vec = []
    for d in dict_list:
        sim = similarity2(curr_list['annotations'], d['annotations'], symmetric = False, use_inter_similarity = False, )
        sim_vec.append(sim)
        sim_val_vec.append(sim[0])
    sim_val_vec
    inds = sorted(range(len(sim_val_vec)),key=lambda x:sim_val_vec[x], reverse = True)
    print 'target name -> %s' % curr_list['short_name']
    #print curr_list['annotations']
    for i in range(0,20):
        print 'match value: %.2f, best match: %s' % (sim_val_vec[inds[i]], dict_list[inds[i]]['short_name'])
        #print dict_list[inds[i]]['annotations']
    print '\n'

In [ ]:
from similarity import similarity, similarity2
from similarity_intra import load_ontologies

# Computes the intra and inter semantic similarity between two neurons
# in: n1@str, n2@str: the two neurons to measure similarity
# out: (score:float, [(matching_properties, explanation@str)])
#similarity('PV Layer 1 cortex acetylcholine neuron', 'Layer 1a striatum gaba fast-spiking neuron')
print similarity('dorsal raphe serotonergic cell', 'Midbrain Slc6a4-expressing serotonergic cells', symmetric = True)
print similarity('Midbrain Slc6a4-expressing serotonergic cells', 'dorsal raphe serotonergic cell', symmetric = False)

In [ ]:
similarity2(gene_dict_list[25]['annotations'], dict_list[25]['annotations'])

In [ ]:
print dict_list[25]['short_name']
print gene_exp_table.iloc[51]['annotations']

In [ ]:
from  more_itertools import unique_everseen

def find_matching_neurons(target_neuron_anns, matching_ann_dict_list, match_thresh = .8):
    sim_vec = []
    sim_val_vec = []
    matching_neuron_list = []
    for d in matching_ann_dict_list:
        sim = similarity2(target_neuron_anns['annotations'], d['annotations'], symmetric = False, use_inter_similarity = False)
        sim_vec.append(sim)
        sim_val_vec.append(sim[0])
    sim_val_vec
    inds = sorted(range(len(sim_val_vec)),key=lambda x:sim_val_vec[x], reverse = True)
    for i in range(0, len(inds)):
        if sim_val_vec[inds[i]] > match_thresh:
            matching_neuron_list.append(dict_list[inds[i]]['neuron_name_orig'])
    if (len(matching_neuron_list) <= 2):
        #print match_thresh, target_neuron_anns
        return find_matching_neurons(target_neuron_anns, matching_ann_dict_list, match_thresh - .05)
    elif match_thresh < .4:
        return []
    return list(unique_everseen(matching_neuron_list))

In [ ]:
ind = 63
#print gene_exp_table.iloc[ind]['neuron_name']
find_matching_neurons(gene_exp_table.iloc[ind], dict_list)

In [ ]:
import numpy as np
# use sherlok to annotate each curated neuron mention in neuroelectro
#annot_vec = []
gene_dict_list = []
for index, row in gene_exp_table.iterrows():
    temp_dict = {}
    if len(row['annotations']) > 0:
        matched_neuron_list = find_matching_neurons(row, dict_list)
    else:
        matched_neuron_list = []
    temp_dict['matched_neuron_list'] = ';'.join(matched_neuron_list)
    temp_dict['Index'] = index
    gene_dict_list.append(temp_dict)

In [ ]:
df_gene = pd.DataFrame.from_records(gene_dict_list, index = 'Index')
gene_exp_table_final = gene_exp_table.join(df_gene)

In [ ]:
gene_exp_table_final.head()

In [ ]:
# work on sorting gene exp rows by semantic similarity

from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import leaders, fclusterdata, linkage, dendrogram, leaves_list

# calculate pairwise distance matrix based on similarity
dist_mat = []
labels = []
for index, row1 in gene_exp_table_final.iterrows():
    sim_vec = []
    ann1 = row1['annotations']
    labels.append(row1['short_name'])
    for i, row2 in gene_exp_table.iterrows():
        ann2 = row2['annotations']
        
        (sim_val, exp) = similarity2(ann1, ann2, symmetric = True, use_inter_similarity = False)
        sim_dist = 1 - sim_val
        sim_vec.append(sim_dist)
    dist_mat.append(sim_vec)
    
# 
d = squareform(dist_mat)
Z = linkage(d)
sorted_rows = leaves_list(Z)

In [ ]:
gene_exp_table_final = gene_exp_table_final.iloc[sorted_rows]
gene_exp_table_final.to_csv('df_gene.csv', sep='\t', encoding='utf-8')

In [ ]:
dendrogram(Z, labels = labels)